In [1]:
#change an excel sheet to a csv file
#import python packages for opening excel and csv files
import xlrd
import csv

#declare a function to open the excel workbook and convert it to a csv file
def csv_from_excel(workbook):
    wb = xlrd.open_workbook(workbook)
    sh = wb.sheet_by_name('Sheet1')
#open the workbook and write each row in the first sheet to a row in the new csv file
    with open('MS.csv', 'w') as MS:
        wr = csv.writer(MS, quoting=csv.QUOTE_ALL)

        for rownum in range(sh.nrows):
            wr.writerow(sh.row_values(rownum))
#close the newly written csv file
    MS.close()
#call the function to convert the excel file containing your output data
csv_from_excel("MS.xlsx")

#read csv file as a dictionary
#write the csv file as dictionary with gene names as keys and 'unique hits' as values, removing the header
import csv
with open ("MS.csv", mode= 'r') as infile:
    reader = csv.reader(infile)
    next(reader,None)
    with open("MS_new.csv", mode= 'w') as outfile: 
        writer = csv.writer(outfile)
        hitnumbers= {rows[3]:rows[0] for rows in reader}

#convert values in dictionary from strings to floats
hitnumbers_ints= dict((k,float(v)) for k,v in hitnumbers.items())

#convert the dictionary into a list containing all unique hit values greater than 1
hits_list=[]
for key, value in hitnumbers_ints.items():
    if value > 1:
        hits_list.append(key)
        
#read the csv file as a dictionary
#write the csv file as dictionary with gene names as keys and molecular weights as values, removing the header
with open ("MS.csv", mode= 'r') as infile:
    reader = csv.reader(infile)
    next(reader,None)
    with open("MS_new.csv", mode= 'w') as outfile: 
        writer = csv.writer(outfile)
        MW= {rows[3]:rows[4] for rows in reader}
        
#convert values from strings to floats
MW_float= dict((k,float(v)) for k,v in MW.items())

#convert the dictionary into a list containing all gene names with molecular weights in the range of 50-70kDa
MW_list=[]
for key, value in MW_float.items():
    if 50.0 < value < 70.0:
        MW_list.append(key)

#concatenate all gene names with greater than 1 unique hits as well as molecular weights in the range of 50-70 into a master list
sig_hits= (list(set(hits_list) & (set(MW_list))))

In [ ]:
#import python packages for parsing HTML
import requests
from bs4 import BeautifulSoup

#declare a function to search filtered gene names from the master list on the uniprot website using a specific keyword
def search_uniprot(genes, keyword):
        keyword = keyword.lower()

        #create an empty list that will be used to write a csv containing all gene names identified by the keyword
        hits = []

        for gene in genes:
            
            gene= gene.lower()
            
            #this is the url from uniprot that ensures the gene name search result is reviewed, is in the organism homo sapiens, and is sorted by top result
            query_url = "https://www.uniprot.org/uniprot/?query=reviewed:yes+AND+organism:9606+" + gene + "&sort=score"
            
            #sends an http request for the search result page for the specific gene name 
            query_page = requests.get(query_url)
            #use beautiful soup to the html from the requested webpage
            query_soup = BeautifulSoup(query_page.text, "html.parser")
            
            #ensures procession if and only if uniprot returns search results 
            if "Sorry, no results found for your search term." not in (str(query_soup.findAll("h2"))):

                #finds the first uniprot identifier in search results for the gene name
                entry_id = str(query_soup.tbody.tr["id"])
                entry_url = "https://www.uniprot.org/uniprot/" + entry_id + ".txt"
                
                #sends an http request for the text version of entry for uniprot identifier
                entry_page = requests.get(entry_url).text

                entry_page = entry_page.lower()
                
                start = entry_page.find("""-!-""")
                
                stop = entry_page.find("copyrighted")
                
                #search uniprot text page using consistent demarcations specifying gene description
                if gene in entry_page[start:stop]:
                    if keyword in entry_page.lower():
                        hits.append(gene)
                        #shows genes that were found with keyword
                        print(gene)

        #writes all genes identified by the keyword to new csv file
        with open(keyword + ".csv", 'w', newline='') as csvfile:
            genewriter = csv.writer(csvfile, delimiter=',', quoting=csv.QUOTE_ALL)
            genewriter.writerow(hits)
#call the function to search filtered list of gene names by a specific keyword
search_uniprot(sig_hits, "acetyltransferase")